In [ ]:
from nlb_tools.nwb_interface import NWBDataset
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rc('font', size=18)

In [ ]:
# foldername = "/Volumes/TOSHIBA EXT/sherry/area2/raeed/"
# import h5py 
monkey = "Han_20171207"
monkey_abbr = 'H'

# f = h5py.File(foldername+ monkey + '_COactpas_TD.mat','r')
# unit_guide = np.array(f['trial_data']['S1_unit_guide'][0])
# np.savez(monkey+'_unit_guide', unit_guide = unit_guide) 
with np.load(monkey+'_unit_guide.npz') as data:
    unit_guide = data['unit_guide']
print(unit_guide)
print(unit_guide.shape)

In [ ]:
foldername = "/Volumes/TOSHIBA EXT/sherry/area2/s1-kinematics/sensory_mapping/"
import pandas as pd
elec_map = pd.read_csv(foldername+'elec_map.csv')
elec_map = elec_map[elec_map['monkey']==monkey_abbr]

rf_map = pd.read_csv(foldername+'rf_map.csv')
rf_map = rf_map[rf_map['monkey']==monkey_abbr]

# print(elec_map)
# print(rf_map)

rf_cutaneous = rf_map[rf_map['modality']=='cutaneous']['label'] #modality to label
rf_deep = rf_map[rf_map['modality']=='deep']['label']
deep_channels = elec_map['chan'][elec_map['label'].isin(rf_deep)].to_numpy() #label to chan
cutaneous_channels = elec_map['chan'][elec_map['label'].isin(rf_cutaneous)].to_numpy()
print(deep_channels.shape)
print(deep_channels)
print(cutaneous_channels.shape)
print(cutaneous_channels)
deep_neurons = []
cutaneous_neurons = []
for i in range(len(unit_guide)): #chan to neuron
    if unit_guide[i] in deep_channels:
        deep_neurons.append(i)
    if unit_guide[i] in cutaneous_channels:
        cutaneous_neurons.append(i)
print(len(deep_neurons))
print(deep_neurons)
print(len(cutaneous_neurons))
print(cutaneous_neurons)

In [ ]:
dropped_nrn = [44,82]
CD_sum_weights = np.zeros([len(unit_guide)])
FB_sum_weights = np.zeros([len(unit_guide)])
for idx in dropped_nrn:
    CD_sum_weights[idx] = -999999
    FB_sum_weights[idx] = -999999
CD_sum_weights

In [ ]:
folder = "/Users/sherryan/area2_population_analysis/"
data = np.load(folder+'Han_20171207'+'_CDFB_weights_pc_proj_back.npz') 
CD_weights = data['CD_axes']
FB_weights = data['FB_axes']

In [ ]:
idx_to_fill = np.argwhere(CD_sum_weights>-999).squeeze()
print(idx_to_fill)
CD_sum_weights_valid = np.sum(abs(CD_weights),axis=0)
FB_sum_weights_vallid = np.sum(abs(FB_weights),axis=0)
print(CD_sum_weights_valid.shape)
for i in range(len(idx_to_fill)):
    CD_sum_weights[idx_to_fill[i]] = CD_sum_weights_valid[i]
    FB_sum_weights[idx_to_fill[i]] = FB_sum_weights_vallid[i]
CD_sum_weights

In [ ]:
array_mat = np.zeros([10,10])
print(array_mat)
fig, ax = plt.subplots()
im = ax.imshow(array_mat)

In [ ]:
# chan to row,col location
deep_loc = np.vstack([elec_map['rowNum'][elec_map['chan'].isin(deep_channels)].to_numpy(),elec_map['colNum'][elec_map['chan'].isin(deep_channels)].to_numpy()]).T-1
cutaneous_loc = np.vstack([elec_map['rowNum'][elec_map['chan'].isin(cutaneous_channels)].to_numpy(),elec_map['colNum'][elec_map['chan'].isin(cutaneous_channels)].to_numpy()]).T-1
deep_chan_map = np.zeros([10,10])
for loc in deep_loc:
    deep_chan_map[loc[0],loc[1]] = 1
cutaneous_chan_map = np.zeros([10,10])
for loc in cutaneous_loc:
    cutaneous_chan_map[loc[0],loc[1]] = 1

In [ ]:
fig, ax = plt.subplots()
im = ax.imshow(np.flip(deep_chan_map,1))
plt.title('Deep map')

In [ ]:
fig, ax = plt.subplots()
im = ax.imshow(np.flip(cutaneous_chan_map,1))
plt.title('Cutaneous map')

In [ ]:
cd_neurons = np.argsort(CD_sum_weights)[-10:]
cd_channels = unit_guide[cd_neurons]
fb_neurons = np.argsort(FB_sum_weights)[-10:]
fb_channels = unit_guide[fb_neurons]
top_cd_loc = np.vstack([elec_map['rowNum'][elec_map['chan'].isin(cd_channels)].to_numpy(),elec_map['colNum'][elec_map['chan'].isin(cd_channels)].to_numpy()]).T-1
top_fb_loc = np.vstack([elec_map['rowNum'][elec_map['chan'].isin(fb_channels)].to_numpy(),elec_map['colNum'][elec_map['chan'].isin(fb_channels)].to_numpy()]).T-1
cd_chan_map = np.zeros([10,10])
for loc in top_cd_loc:
    cd_chan_map[loc[0],loc[1]] = 1
fb_chan_map = np.zeros([10,10])
for loc in top_fb_loc:
    fb_chan_map[loc[0],loc[1]] = 1

In [ ]:
fig, ax = plt.subplots()
im = ax.imshow(np.flip(cd_chan_map,1))
plt.title('CD map')

In [ ]:
fig, ax = plt.subplots()
im = ax.imshow(np.flip(fb_chan_map,1))
plt.title('FB map')

In [ ]:
cut_total_prop = len(cutaneous_neurons)/len(unit_guide)
deep_total_prop = len(deep_neurons)/len(unit_guide)
print(cut_total_prop,deep_total_prop)

In [ ]:
top_nrn_range = np.arange(10, 151, 10)
cd_cut_deep_prop = np.zeros([len(top_nrn_range),2])
for i in range(len(top_nrn_range)):
    n = top_nrn_range[i]
    cd_cut_intersect = set(cutaneous_neurons).intersection(set(np.argsort(CD_sum_weights)[-n:]) )
    # print(cd_cut_intersect)
    cd_deep_intersect = set(deep_neurons).intersection(set(np.argsort(CD_sum_weights)[-n:]) )
    # print(cd_deep_intersect)
    print(len(cd_cut_intersect)/n, len(cd_deep_intersect)/n)
    cd_cut_deep_prop[i] = [len(cd_cut_intersect)/n, len(cd_deep_intersect)/n]

In [ ]:
plt.plot(top_nrn_range,cd_cut_deep_prop[:,0],color = 'blue',label = 'cutaneous')
plt.plot(top_nrn_range,cd_cut_deep_prop[:,1],color = 'orange',label = 'deep')
plt.xlabel('Top neuron #')
plt.ylabel('Proportion')
plt.title("CD neuron distribution")
plt.axhline(cut_total_prop, color = 'blue',linestyle='--')
plt.axhline(deep_total_prop, color = 'orange', linestyle='--')
plt.ylim([0.25,0.95])

plt.legend()

In [ ]:
fb_cut_deep_prop = np.zeros([len(top_nrn_range),2])
for i in range(len(top_nrn_range)):
    n = top_nrn_range[i]
    fb_cut_intersect = set(cutaneous_neurons).intersection(set(np.argsort(FB_sum_weights)[-n:]) )
    # print(fb_cut_intersect)
    fb_deep_intersect = set(deep_neurons).intersection(set(np.argsort(FB_sum_weights)[-n:]))
    # print(fb_deep_intersect)
    print(len(fb_cut_intersect)/n, len(fb_deep_intersect)/n)
    fb_cut_deep_prop[i] = [len(fb_cut_intersect)/n, len(fb_deep_intersect)/n]

In [ ]:
plt.plot(top_nrn_range,fb_cut_deep_prop[:,0],color = 'blue',label = 'cutaneous')
plt.plot(top_nrn_range,fb_cut_deep_prop[:,1],color = 'orange',label = 'deep')
plt.xlabel('Top neuron #')
plt.ylabel('Proportion')
plt.title("FB neuron distribution")
plt.axhline(cut_total_prop, color = 'blue',linestyle='--')
plt.axhline(deep_total_prop, color = 'orange', linestyle='--')
# plt.ylim([0.25,0.95])
plt.legend()

In [ ]:
distal_parts = ['hand','wrist','forearm']
proximal_parts = ['elbow','humerus','shoulder','torso','arm']

rf_distal = rf_map[rf_map['receptive_field'].isin(distal_parts)]['label']
rf_proximal = rf_map[rf_map['receptive_field'].isin(proximal_parts)]['label']
distal_channels = elec_map['chan'][elec_map['label'].isin(rf_distal)].to_numpy()
proximal_channels = elec_map['chan'][elec_map['label'].isin(rf_proximal)].to_numpy()
print(distal_channels.shape)
print(distal_channels)
print(proximal_channels.shape)
print(proximal_channels)
distal_neurons = []
proximal_neurons = []
for i in range(len(unit_guide)):
    if unit_guide[i] in distal_channels:
        distal_neurons.append(i)
    if unit_guide[i] in proximal_channels:
        proximal_neurons.append(i)
print(len(distal_neurons))
print(distal_neurons)
print(len(proximal_neurons))
print(proximal_neurons)

In [ ]:
dist_total_prop = len(distal_neurons)/len(unit_guide)
prox_total_prop = len(proximal_neurons)/len(unit_guide)
print(dist_total_prop,prox_total_prop)

In [ ]:
cd_dist_prox_prop = np.zeros([len(top_nrn_range),2])
for i in range(len(top_nrn_range)):
    n = top_nrn_range[i]
    cd_dist_intersect = set(distal_neurons).intersection(set(np.argsort(CD_sum_weights)[-n:]) )
    cd_prox_intersect = set(proximal_neurons).intersection(set(np.argsort(CD_sum_weights)[-n:]))
    print(len(cd_dist_intersect)/n, len(cd_prox_intersect)/n)
    cd_dist_prox_prop[i] = [len(cd_dist_intersect)/n, len(cd_prox_intersect)/n]

plt.plot(top_nrn_range,cd_dist_prox_prop[:,0],color = 'green',label = 'Distal')
plt.plot(top_nrn_range,cd_dist_prox_prop[:,1],color = 'red',label = 'Proximal')
plt.xlabel('Top neuron #')
plt.ylabel('Proportion')
plt.title("CD neuron distribution")
plt.axhline(dist_total_prop, color = 'green',linestyle='--')
plt.axhline(prox_total_prop, color = 'red', linestyle='--')
plt.ylim([0.25,0.95])
plt.legend()

In [ ]:
fb_dist_prox_prop = np.zeros([len(top_nrn_range),2])
for i in range(len(top_nrn_range)):
    n = top_nrn_range[i]
    fb_dist_intersect = set(distal_neurons).intersection(set(np.argsort(FB_sum_weights)[-n:]) )
    fb_prox_intersect = set(proximal_neurons).intersection(set(np.argsort(FB_sum_weights)[-n:]))
    print(len(fb_dist_intersect)/n, len(fb_prox_intersect)/n)
    fb_dist_prox_prop[i] = [len(fb_dist_intersect)/n, len(fb_prox_intersect)/n]

plt.plot(top_nrn_range,fb_dist_prox_prop[:,0],color = 'green',label = 'Distal')
plt.plot(top_nrn_range,fb_dist_prox_prop[:,1],color = 'red',label = 'Proximal')
plt.xlabel('Top neuron #')
plt.ylabel('Proportion')
plt.title("FB neuron distribution")
plt.axhline(dist_total_prop, color = 'green',linestyle='--')
plt.axhline(prox_total_prop, color = 'red', linestyle='--')
plt.ylim([0.25,0.95])
plt.legend()

In [ ]:
monkey = 'Duncan_20190710'
# monkey = 'Han_20171207'
# monkey = 'Chips_20170913'
# monkey = "Lando_20170731"

monkey = 'Duncan_20191106'


foldername = "~/area2_population_analysis/s1-kinematics/actpas_NWB/"
# filename = foldername + monkey + "_COactpas_reformat.nwb"
filename = foldername + monkey + '_COactpas_TD_offset6.nwb'
dataset = NWBDataset(filename, split_heldout=False)
dataset.resample(10)
dataset.bin_width
speed = np.sqrt(np.sum(dataset.data['hand_vel'][:].T**2,axis=0)).to_numpy().reshape((-1,1))
dataset.add_continuous_data(speed,'speed')

In [ ]:
from Area2_analysis.lr_funcs import nans
#make dictionary for trial condition (reaching directions) for Stratified CV
active_mask = (dataset.trial_info.ctr_hold_bump==0) & (dataset.trial_info.split != 'none')
passive_mask = (dataset.trial_info.ctr_hold_bump==1) & (dataset.trial_info.split != 'none')
nan_mask = (np.isnan(dataset.trial_info.ctr_hold_bump)) & (dataset.trial_info.split != 'none')

trial_mask = active_mask
active_trials_idx = np.array(dataset.trial_info.loc[trial_mask]['trial_id'])
active_n_trials = dataset.trial_info.loc[trial_mask].shape[0]
print(active_n_trials,'active trials')

trial_mask = passive_mask
passive_trials_idx = np.array(dataset.trial_info.loc[trial_mask]['trial_id'])
passive_n_trials = dataset.trial_info.loc[trial_mask].shape[0]
print(passive_n_trials,'passive trials')

trial_mask = nan_mask
nan_trials_idx = np.array(dataset.trial_info.loc[trial_mask]['trial_id'])
nan_n_trials = dataset.trial_info.loc[trial_mask].shape[0]
print(nan_n_trials,'reach-bump trials')


active_cond_dir_idx = []
passive_cond_dir_idx = []
nan_cond_dir_idx = []
for direction in [0,45,90,135,180,225,270,315]:
# for direction in [0,90,180,270]:
    active_cond_dir_idx.append(np.where((dataset.trial_info['cond_dir'] == direction) & (dataset.trial_info['ctr_hold_bump'] == 0) & \
           (dataset.trial_info['split'] != 'none'))[0])
    passive_cond_dir_idx.append(np.where((dataset.trial_info['cond_dir']%360 == direction) & (dataset.trial_info['ctr_hold_bump'] == 1) & \
           (dataset.trial_info['split'] != 'none'))[0])
    nan_cond_dir_idx.append(np.where((dataset.trial_info['cond_dir']%360 == direction) & (np.isnan(dataset.trial_info['ctr_hold_bump'])) & \
           (dataset.trial_info['split'] != 'none'))[0])
    
active_cond_dict = nans([active_n_trials])
i = 0
for idx in active_trials_idx:
    for cond in range(0,len(active_cond_dir_idx)):
        if idx in active_cond_dir_idx[cond]:
            active_cond_dict[i] = cond
            break
    i+=1
print(active_cond_dict)
print(len(active_cond_dict))

passive_cond_dict = nans([passive_n_trials])
i = 0
for idx in passive_trials_idx:
    for cond in range(0,len(passive_cond_dir_idx)):
        if idx in passive_cond_dir_idx[cond]:
            passive_cond_dict[i] = cond
            break
    i+=1
print(passive_cond_dict)
print(len(passive_cond_dict))

nan_cond_dict = nans([nan_n_trials])
i = 0
for idx in nan_trials_idx:
    for cond in range(0,len(nan_cond_dir_idx)):
        if idx in nan_cond_dir_idx[cond]:
            nan_cond_dict[i] = cond
            break
    i+=1
print(nan_cond_dict)
print(len(nan_cond_dict))

In [ ]:
dataset.data

In [ ]:

plot_range = (-350, 650)
x_axis = np.arange(plot_range[0], plot_range[1], dataset.bin_width)
passive_df = dataset.make_trial_data(align_field='move_onset_time', align_range=plot_range, ignored_trials=~passive_mask)
active_df = dataset.make_trial_data(align_field='move_onset_time', align_range=plot_range, ignored_trials=~active_mask)
nan_df = dataset.make_trial_data(align_field='move_onset_time', align_range=plot_range, ignored_trials=~nan_mask)

fig, ax = plt.subplots(figsize=(8,6))
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
for _, trial in active_df.groupby('trial_id'):
    plt.plot(x_axis,trial.speed, color='k', linewidth=0.5)
for _, trial in nan_df.groupby('trial_id'):
    plt.plot(x_axis, trial.speed, color='orange', linewidth=0.5)
for _, trial in passive_df.groupby('trial_id'):
    plt.plot(x_axis, trial.speed, color='red', linewidth=0.5)
plt.xlabel('Time after movement onset (ms)')
plt.ylabel('Hand speed (cm/s)')
plt.axvline(0, color = 'k',linestyle = '--')
# plt.title('Speed aligned to move_onset')
plt.xlim([-300,600])
plt.ylim([0,80])
plt.axvline(120, color = 'k', linestyle = '--')

plt.tight_layout()
figDir = '/Users/sherryan/Desktop/paper/'
plt.savefig(figDir + monkey + '_speed.pdf',dpi = 'figure')

In [ ]:

plot_range = (-350, 650)
x_axis = np.arange(plot_range[0], plot_range[1], dataset.bin_width)
passive_df = dataset.make_trial_data(align_field='move_onset_time', align_range=plot_range, ignored_trials=~passive_mask)
active_df = dataset.make_trial_data(align_field='move_onset_time', align_range=plot_range, ignored_trials=~active_mask)
nan_df = dataset.make_trial_data(align_field='move_onset_time', align_range=plot_range, ignored_trials=~nan_mask)

fig, ax = plt.subplots(figsize=(8,6))
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
for _, trial in active_df.groupby('trial_id'):
    plt.plot(x_axis,trial['hand_vel']['x'], color='k', linewidth=0.5)
for _, trial in nan_df.groupby('trial_id'):
    plt.plot(x_axis, trial['hand_vel']['x'], color='orange', linewidth=0.5)
# for _, trial in passive_df.groupby('trial_id'):
#     plt.plot(x_axis, trial['hand_vel']['x'], color='red', linewidth=0.5)
plt.xlabel('Time after movement onset (ms)')
plt.ylabel('Hand speed (cm/s)')
plt.axvline(0, color = 'k',linestyle = '--')
# plt.title('Speed aligned to move_onset')
plt.xlim([-300,600])
plt.ylim([-45,45])
plt.axvline(120, color = 'k', linestyle = '--')

plt.tight_layout()
figDir = '/Users/sherryan/Desktop/paper/'
# plt.savefig(figDir + monkey + '_speed.pdf',dpi = 'figure')

In [ ]:

plot_range = (-350, 650)
x_axis = np.arange(plot_range[0], plot_range[1], dataset.bin_width)
passive_df = dataset.make_trial_data(align_field='move_onset_time', align_range=plot_range, ignored_trials=~passive_mask)
active_df = dataset.make_trial_data(align_field='move_onset_time', align_range=plot_range, ignored_trials=~active_mask)
nan_df = dataset.make_trial_data(align_field='move_onset_time', align_range=plot_range, ignored_trials=~nan_mask)

fig, ax = plt.subplots(figsize=(8,6))
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
for _, trial in active_df.groupby('trial_id'):
    plt.plot(x_axis,trial['hand_vel']['x'], color='k', linewidth=0.5)
for _, trial in nan_df.groupby('trial_id'):
    plt.plot(x_axis, trial['hand_vel']['x'], color='orange', linewidth=0.5)
# for _, trial in passive_df.groupby('trial_id'):
#     plt.plot(x_axis, trial['hand_vel']['x'], color='red', linewidth=0.5)
plt.xlabel('Time after movement onset (ms)')
plt.ylabel('Hand speed (cm/s)')
plt.axvline(0, color = 'k',linestyle = '--')
# plt.title('Speed aligned to move_onset')
plt.xlim([-300,600])
plt.ylim([-45,45])
plt.axvline(120, color = 'k', linestyle = '--')

plt.tight_layout()
figDir = '/Users/sherryan/Desktop/paper/'
# plt.savefig(figDir + monkey + '_speed.pdf',dpi = 'figure')

In [ ]:
passive_df = dataset.make_trial_data(align_field='move_onset_time', align_range=(-350,550), ignored_trials=~passive_mask)
active_df = dataset.make_trial_data(align_field='move_onset_time', align_range=(-350,550), ignored_trials=~active_mask)


In [ ]:
nan_cond_dir_idx

In [ ]:
check manipulandum 
force after pertutrbation
Han vs. Chips
then Han vs. Duncan

In [ ]:
dataset.data['force']

In [ ]:
nan_trial_info = dataset.trial_info.loc[nan_mask].copy()
nan_trials_idx = nan_trial_info['trial_id'].to_numpy()
nan_trial_info.head()

In [ ]:
nan_mask.sum()

In [ ]:
mask = nan_mask  

dt_move = (
    dataset.trial_info.loc[mask, 'move_onset_time']
    - dataset.trial_info.loc[mask, 'bump_time']
).dt.total_seconds() * 1000

dt_go = (
    dataset.trial_info.loc[mask, 'go_cue_time']
    - dataset.trial_info.loc[mask, 'bump_time']
).dt.total_seconds() * 1000

In [ ]:
dt_go_move = (
    dataset.trial_info.loc[mask, 'go_cue_time']
    - dataset.trial_info.loc[mask, 'move_onset_time']
).dt.total_seconds() * 1000

In [ ]:
np.mean(~np.isnan(dt_go_move.to_numpy()))

In [ ]:
passive_df = dataset.make_trial_data(align_field='bump_time', align_range=(-100,500), ignored_trials=~passive_mask)

In [ ]:
plot_range = (-100,500)
x_axis = np.arange(plot_range[0], plot_range[1], dataset.bin_width)
var = 'force'
fig, ax = plt.subplots(figsize=(10,6))
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
count = 0
for _, trial in passive_df.groupby('trial_id'):
    if len(trial[var]) == len(x_axis):
        plt.plot(x_axis,trial[var]['x'], color='k', linewidth=0.5)
        count+=1
print(count,'passive trials')
# count = 0
# for _, trial in passive_df.groupby('trial_id'):
#     if len(trial[var]) == len(x_axis):
#         plt.plot(x_axis, trial[var], color='red', linewidth=0.5)
#         count+=1
# print(count,'passive trials')

In [ ]:
for _, trial in passive_df.groupby('trial_id'):
    if len(trial[var]) == len(x_axis):
        plt.plot(x_axis,trial[var]['y'], color='k', linewidth=0.5)
        count+=1
print(count,'passive trials')

In [ ]:
plot_range = (-100,500)
x_axis = np.arange(plot_range[0], plot_range[1], dataset.bin_width)
var = 'hand_vel'
fig, ax = plt.subplots(figsize=(10,6))
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
count = 0
for _, trial in passive_df.groupby('trial_id'):
    if len(trial[var]) == len(x_axis):
        plt.plot(x_axis,trial[var]['y'], color='k', linewidth=0.5)
        count+=1
print(count,'passive trials')
# count = 0
# for _, trial in passive_df.groupby('trial_id'):
#     if len(trial[var]) == len(x_axis):
#         plt.plot(x_axis, trial[var], color='red', linewidth=0.5)
#         count+=1
# print(count,'passive trials')

In [ ]:
for _, trial in passive_df.groupby('trial_id'):
    if len(trial[var]) == len(x_axis):
        plt.plot(x_axis,trial[var]['y'], color='k', linewidth=0.5)
        count+=1
print(count,'passive trials')

In [ ]:
passive_df = dataset.make_trial_data(align_field='bump_time', align_range=(-200,300), ignored_trials=~passive_mask)

In [ ]:
plot_range = (-200,300)
x_axis = np.arange(plot_range[0], plot_range[1], dataset.bin_width)
var = 'hand_vel'
fig, ax = plt.subplots(figsize=(10,6))
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
count = 0
for _, trial in passive_df.groupby('trial_id'):
    if len(trial[var]) == len(x_axis):
        plt.plot(x_axis,trial[var]['x'], color='k', linewidth=0.5)
        count+=1
print(count,'passive trials')
# count = 0
# for _, trial in passive_df.groupby('trial_id'):
#     if len(trial[var]) == len(x_axis):
#         plt.plot(x_axis, trial[var], color='red', linewidth=0.5)
#         count+=1
# print(count,'passive trials')

In [ ]:
for _, trial in passive_df.groupby('trial_id'):
    if len(trial[var]) == len(x_axis):
        plt.plot(x_axis,trial[var]['y'], color='k', linewidth=0.5)
        count+=1
print(count,'passive trials')

In [ ]:
for _, trial in passive_df.groupby('trial_id'):
    if len(trial[var]) == len(x_axis):
        plt.plot(x_axis,trial[var]['xmo'], color='k', linewidth=0.5)
        count+=1
print(count,'passive trials')

In [ ]:
nan_df.force

In [ ]:
dt_go_move

In [ ]:
plt.hist(dt_go_move,   bins=90, alpha=0.6, label='Go cue to Move')


In [ ]:
plt.hist(dt_go,   bins=90, alpha=0.6, label='Go cue')


In [ ]:
plt.figure(figsize=(6,4))

plt.hist(dt_move, bins=90, alpha=0.6, label='Move onset')
# plt.hist(dt_go,   bins=90, alpha=0.6, label='Go cue')

plt.axvline(0, color='k', linestyle='--')
plt.xlabel("Time (ms): event − bump")
plt.ylabel("Count")
plt.legend(frameon=False)
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
plt.hist((dataset.trial_info.move_onset_time - dataset.trial_info.bump_time).dt.total_seconds()*1000,90)
# plt.xlim(-100,100)

In [ ]:
n_neurons = dataset.data.spikes.shape[1]
print(n_neurons,'neurons')

plot_range = (-100, 100)
x_axis = np.arange(plot_range[0], plot_range[1], dataset.bin_width)
passive_df = dataset.make_trial_data(align_field='move_onset_time', align_range=plot_range, ignored_trials=~passive_mask)
active_df = dataset.make_trial_data(align_field='move_onset_time', align_range=plot_range, ignored_trials=~active_mask)
nan_df = dataset.make_trial_data(align_field='move_onset_time', align_range=plot_range, ignored_trials=~nan_mask)

active_trials_spikes = []
for _, trial in active_df.groupby('trial_id'):
    active_trials_spikes.append(np.sum(trial.spikes,axis=1))
passive_trials_spikes = []
for _, trial in passive_df.groupby('trial_id'):
    passive_trials_spikes.append(np.sum(trial.spikes,axis=1))
nan_trials_spikes = []
for _, trial in nan_df.groupby('trial_id'):
    nan_trials_spikes.append(np.sum(trial.spikes,axis=1))

fig, ax = plt.subplots(figsize=(8,6))
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.plot(x_axis,np.sum(active_trials_spikes,axis = 0)/dataset.bin_width*1000/len(active_trials_spikes)/n_neurons,"-o",color = 'k',label = 'Active')
plt.plot(x_axis,np.sum(passive_trials_spikes,axis = 0)/dataset.bin_width*1000/len(passive_trials_spikes)/n_neurons,"-o",color = 'red',label = 'Passive')
plt.plot(x_axis,np.sum(nan_trials_spikes,axis = 0)/dataset.bin_width*1000/len(nan_trials_spikes)/n_neurons,"-o",label = 'Reach-bump')


plt.legend()

# plt.title('Peristimulus aligned to move_onset')
plt.ylabel('Average Firing Rate (sp/s)')
plt.xlabel('Time after movement onset (ms)')
plt.axvline(0, color = 'k',linestyle = '--')    

plt.xlim([-100,100])
plt.tight_layout()
plt.show()
# plt.savefig(figDir + monkey + '_psth.pdf',dpi = 'figure')

In [ ]:
active_deep_spikes = []
active_cutaneous_spikes = []

for _, trial in active_df.groupby('trial_id'):
    active_deep_spikes.append(np.sum(trial.spikes.to_numpy()[:,deep_neurons],axis=1))
    active_cutaneous_spikes.append(np.sum(trial.spikes.to_numpy()[:,cutaneous_neurons],axis=1))
fig, ax = plt.subplots(figsize=(8,6))
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.plot(x_axis,np.sum(active_deep_spikes,axis = 0)/dataset.bin_width*1000/len(active_deep_spikes)/n_neurons,"-o",color='green',label = 'Deep')
plt.plot(x_axis,np.sum(active_cutaneous_spikes,axis = 0)/dataset.bin_width*1000/len(active_cutaneous_spikes)/n_neurons,"-o",color='orange',label = 'Cutaneous')
plt.legend()
plt.title('Active')
plt.ylabel('Average Firing Rate (sp/s)')
plt.xlabel('Time after movement onset (ms)')
plt.axvline(0, color = 'k',linestyle = '--')    

plt.xlim([-100,100])
plt.tight_layout()
plt.show()


In [ ]:
passive_deep_spikes = []
pasisive_cutaneous_spikes = []

for _, trial in passive_df.groupby('trial_id'):
    passive_deep_spikes.append(np.sum(trial.spikes.to_numpy()[:,deep_neurons],axis=1))
    pasisive_cutaneous_spikes.append(np.sum(trial.spikes.to_numpy()[:,cutaneous_neurons],axis=1))
fig, ax = plt.subplots(figsize=(8,6))
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.plot(x_axis,np.sum(passive_deep_spikes,axis = 0)/dataset.bin_width*1000/len(passive_deep_spikes)/n_neurons,"-o",color='green',label = 'Deep')
plt.plot(x_axis,np.sum(pasisive_cutaneous_spikes,axis = 0)/dataset.bin_width*1000/len(pasisive_cutaneous_spikes)/n_neurons,"-o",color='orange',label = 'Cutaneous')
plt.legend()
plt.title('Passive')
plt.ylabel('Average Firing Rate (sp/s)')
plt.xlabel('Time after movement onset (ms)')
plt.axvline(0, color = 'k',linestyle = '--')    

plt.xlim([-100,100])
plt.tight_layout()
plt.show()


In [ ]:
active_distal_spikes = []
active_proximal_spikes = []

for _, trial in active_df.groupby('trial_id'):
    active_distal_spikes.append(np.sum(trial.spikes.to_numpy()[:,distal_neurons],axis=1))
    active_proximal_spikes.append(np.sum(trial.spikes.to_numpy()[:,proximal_neurons],axis=1))
fig, ax = plt.subplots(figsize=(8,6))
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.plot(x_axis,np.sum(active_distal_spikes,axis = 0)/dataset.bin_width*1000/len(active_distal_spikes)/n_neurons,"-o",color = 'purple',label = 'Distal')
plt.plot(x_axis,np.sum(active_proximal_spikes,axis = 0)/dataset.bin_width*1000/len(active_proximal_spikes)/n_neurons,"-o",color = 'pink',label = 'Proximal')
plt.legend()
plt.title('Active')
plt.ylabel('Average Firing Rate (sp/s)')
plt.xlabel('Time after movement onset (ms)')
plt.axvline(0, color = 'k',linestyle = '--')    

plt.xlim([-100,100])
plt.tight_layout()
plt.show()


In [ ]:
passive_distal_spikes = []
passive_proximal_spikes = []

for _, trial in passive_df.groupby('trial_id'):
    passive_distal_spikes.append(np.sum(trial.spikes.to_numpy()[:,distal_neurons],axis=1))
    passive_proximal_spikes.append(np.sum(trial.spikes.to_numpy()[:,proximal_neurons],axis=1))
fig, ax = plt.subplots(figsize=(8,6))
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.plot(x_axis,np.sum(passive_distal_spikes,axis = 0)/dataset.bin_width*1000/len(passive_distal_spikes)/n_neurons,"-o",color = 'purple',label = 'Distal')
plt.plot(x_axis,np.sum(passive_proximal_spikes,axis = 0)/dataset.bin_width*1000/len(passive_proximal_spikes)/n_neurons,"-o",color = 'pink',label = 'Proximal')
plt.legend()
plt.title('Passive')
plt.ylabel('Average Firing Rate (sp/s)')
plt.xlabel('Time after movement onset (ms)')
plt.axvline(0, color = 'k',linestyle = '--')    

plt.xlim([-100,100])
plt.tight_layout()
plt.show()


In [ ]:
# force - forces and torques applied to the manipulandum. x, y, and z are the forces applied in their respective directions, while xmo, ymo, and zmo are the torques in those directions
# hand_pos - x and y position of the hand, in cm
# hand_vel - x and y velocity of the hand, in cm/s
# joint_ang - angle of various monkey arm joints, in degrees
# joint_vel - velocity of various monkey arm joints, in degrees/s
# muscle_len - length of various monkey arm muscles, in m
# muscle_vel - velocity of various monkey arm muscles, in m/s
# spikes - spike trains binned at 1 ms

In [ ]:
dataset.data.keys().unique(0)

In [ ]:
dataset.data['muscle_len']

In [ ]:
dataset.data['muscle_vel']

In [ ]:
dataset.data['joint_ang']

In [ ]:
dataset.data['joint_vel']
# upper_joints = ['elbow_flexion','shoulder_flexion','shoulder_rotation','shoulder_adduction']
# lower_joints = ['radial_pronation','wrist_abduction','wrist_flexion']

In [ ]:
dataset.data.spikes

In [ ]:
# trial_id - a number assigned to each trial during loading
# start_time - time when the trial begins
# end_time - time when the trial ends
# result - whether a trial was rewarded (R), aborted (A), incomplete (I), or failed (F)
# ctr_hold - the amount of time the monkey had to hold in the center before the reach
# ctr_hold_bump - whether there was a bump during the center hold period
# bump_dir - direction of the bump, in degrees. If there was no bump, bump_dir is NaN
# target_on_time - time of target presentation
# target_dir - direction to the target, in degrees
# go_cue_time - time of go cue
# bump_time - time of center hold bump, if there was one
# move_time - time of movement onset, either in response to the go cue for active trials or the center hold bump for passive trials
# cond_dir - bump_dir for passive trials and target_dir for active trials. Though it is redundant with information already in other fields, it is provided for convenience when filtering trials

In [ ]:
dataset.trial_info.keys().unique(0)

In [ ]:
active_mask = (~dataset.trial_info.ctr_hold_bump) & (dataset.trial_info.split != 'none')
passive_mask = (dataset.trial_info.ctr_hold_bump) & (dataset.trial_info.split != 'none')


trial_mask = active_mask
n_trials = dataset.trial_info.loc[trial_mask].shape[0]
print(n_trials,'active trials')

trial_mask = passive_mask
n_trials = dataset.trial_info.loc[trial_mask].shape[0]
print(n_trials,'passive trials')

In [ ]:
passive_mask

In [ ]:
dataset.trial_info.move_onset_time[1:]

In [ ]:
dataset.trial_info.move_onset_time[:-1]

In [ ]:
dataset.trial_info.move_onset_time

In [ ]:
dataset.trial_info.bump_time

In [ ]:
plt.hist(dataset.trial_info.move_onset_time.dt.total_seconds()*1000,90)
plt.xlim(-100,100)

In [ ]:
import pandas as pd
plt.hist((dataset.trial_info.move_onset_time - dataset.trial_info.bump_time).dt.total_seconds()*1000,90)
plt.xlim(-100,100)

In [ ]:
dataset.trial_info.cond_dir.unique()

In [ ]:
dataset.trial_info.split.unique()

In [ ]:
dataset.descriptions

In [ ]:
dataset.bin_width

In [ ]:
dataset.resample(10)
print(dataset.bin_width)

In [ ]:
len(dataset.data.force.y)

In [ ]:
sum(np.isnan(dataset.data.force.y))

In [ ]:
cond_data.hand_pos

In [ ]:
sum(np.isnan(cond_data.hand_pos.x))

In [ ]:
dataset.trial_info['ctr_hold_bump']

In [ ]:
cond_mask = (np.isnan(dataset.trial_info['ctr_hold_bump'])) & \
            (dataset.trial_info['bump_dir']%360==cond[1]) & \
            (dataset.trial_info.split != 'none')
sum(cond_mask)

In [ ]:
cond_mask

In [ ]:
dataset.make_trial_data(align_field='move_onset_time', align_range=(0, 1500), ignored_trials=~cond_mask)

In [ ]:
# All 16 conditions, in the format (ctr_hold_bump, cond_dir)
unique_conditions = [(False, 0.0), (False, 45.0), (False, 90.0), (False, 135.0),
                     (False, 180.0), (False, 225.0), (False, 270.0), (False, 315.0)]
# unique_conditions = [(True, 0.0), (True, 45.0), (True, 90.0), (True, 135.0),
#                      (True, 180.0), (True, 225.0), (True, 270.0), (True, 315.0)]

# unique_conditions = [(np.nan, 0.0), (np.nan, 45.0), (np.nan, 90.0), (np.nan, 135.0),
#                      (np.nan, 180.0), (np.nan, 225.0), (np.nan, 270.0), (np.nan, 315.0)]

# unique_conditions = [(False, 0.0), (False, 90.0), 
#                      (False, 180.0), (False, 270.0)]
# # unique_conditions = [(True, 0.0),(True, 90.0), 
#                      (True, 180.0), (True, 270.0)]

# Initialize figure
fig = plt.figure(figsize=(8, 4))
ax_0 = fig.add_subplot(1,2,1)
ax_1 = fig.add_subplot(1,2,2)

cmap = plt.get_cmap('coolwarm',len(unique_conditions))
custom_palette = [matplotlib.colors.rgb2hex(cmap(i)) for i in range(len(unique_conditions))]
j=0
for cond in unique_conditions:
    # Filter out invalid trials (labeled 'none') and trials in other conditions
    cond_mask = (dataset.trial_info['ctr_hold_bump']==cond[0]) & \
                (dataset.trial_info['cond_dir']%360==cond[1]) & \
                (dataset.trial_info.split != 'none')
# cond_mask = (np.isnan(dataset.trial_info['ctr_hold_bump'])) & \

    # Extract relevant portion of selected trials

    cond_data = dataset.make_trial_data(align_field='move_onset_time', align_range=(0, 1500), ignored_trials=~cond_mask)
    # Plot reaches on appropriate subplot
    for idx, trial in cond_data.groupby('trial_id'):
        ax_0.plot(trial.hand_pos.x, trial.hand_pos.y, color=custom_palette[j], linewidth=0.5)
    j+=1

unique_conditions = [(True, 0.0), (True, 45.0), (True, 90.0), (True, 135.0),
                     (True, 180.0), (True, 225.0), (True, 270.0), (True, 315.0)]
j=0
for cond in unique_conditions:
    cond_mask = (dataset.trial_info['ctr_hold_bump']==cond[0]) & \
            (dataset.trial_info['cond_dir']%360==cond[1]) & \
            (dataset.trial_info.split != 'none')
    cond_data = dataset.make_trial_data(align_field='move_onset_time', align_range=(0, 500), ignored_trials=~cond_mask)
    # Plot reaches on appropriate subplot
    for idx, trial in cond_data.groupby('trial_id'):
        ax_1.plot(trial.hand_pos.x, trial.hand_pos.y, color=custom_palette[j], linewidth=0.5)
    j+=1

    
            
# Add labels
ax_0.set_title('active')
ax_1.set_title('passive')

ax_0.axis("off")
ax_1.axis("off")


figDir = '/Users/sherryan/area2_population_analysis/sfn/'
plt.tight_layout()
plt.savefig(figDir + monkey + '_traj.pdf',dpi = 'figure')
plt.show()

In [ ]:
# All 16 conditions, in the format (ctr_hold_bump, cond_dir)
unique_conditions = [(False, 0.0), (False, 45.0), (False, 90.0), (False, 135.0),
                     (False, 180.0), (False, 225.0), (False, 270.0), (False, 315.0)]
# unique_conditions = [(True, 0.0), (True, 45.0), (True, 90.0), (True, 135.0),
#                      (True, 180.0), (True, 225.0), (True, 270.0), (True, 315.0)]

# unique_conditions = [(np.nan, 0.0), (np.nan, 45.0), (np.nan, 90.0), (np.nan, 135.0),
#                      (np.nan, 180.0), (np.nan, 225.0), (np.nan, 270.0), (np.nan, 315.0)]

# unique_conditions = [(False, 0.0), (False, 90.0), 
#                      (False, 180.0), (False, 270.0)]
# # unique_conditions = [(True, 0.0),(True, 90.0), 
#                      (True, 180.0), (True, 270.0)]

# Initialize figure
fig = plt.figure(figsize=(20, 4))
ax_0 = fig.add_subplot(1,5,1)
ax_1 = fig.add_subplot(1, 5,2)
ax_2 = fig.add_subplot(1, 5,3)
ax_3 = fig.add_subplot(1, 5,4)
ax_4 = fig.add_subplot(1, 5,5)


for cond in unique_conditions:
    # Filter out invalid trials (labeled 'none') and trials in other conditions
    # cond_mask = (dataset.trial_info['ctr_hold_bump']==cond[0]) & \
    cond_mask = (np.isnan(dataset.trial_info['ctr_hold_bump'])) & \
                (dataset.trial_info['cond_dir']%360==cond[1]) & \
                (dataset.trial_info.split != 'none')
# cond_mask = (np.isnan(dataset.trial_info['ctr_hold_bump'])) & \

    # Extract relevant portion of selected trials

    cond_data = dataset.make_trial_data(align_field='move_onset_time', align_range=(-500, 0), ignored_trials=~cond_mask)
    # Plot reaches on appropriate subplot
    for idx, trial in cond_data.groupby('trial_id'):
        ax_0.plot(trial.hand_pos.x, trial.hand_pos.y, color=plt.cm.hsv(cond[1] / 360), linewidth=0.5)
        ax_0.set_xlim([-15, 15])
        ax_0.set_ylim([-15, 15])

    cond_data = dataset.make_trial_data(align_field='move_onset_time', align_range=(0, 500), ignored_trials=~cond_mask)
    # Plot reaches on appropriate subplot
    for idx, trial in cond_data.groupby('trial_id'):
        ax_1.plot(trial.hand_pos.x, trial.hand_pos.y, color=plt.cm.hsv(cond[1] / 360), linewidth=0.5)
        ax_1.set_xlim([-15, 15])
        ax_1.set_ylim([-15, 15])
    
    cond_data = dataset.make_trial_data(align_field='move_onset_time', align_range=(500, 1000), ignored_trials=~cond_mask)
    for idx, trial in cond_data.groupby('trial_id'):
        ax_2.plot(trial.hand_pos.x, trial.hand_pos.y, color=plt.cm.hsv(cond[1] / 360), linewidth=0.5)
        ax_2.set_xlim([-15, 15])
        ax_2.set_ylim([-15, 15])
            
    cond_data = dataset.make_trial_data(align_field='move_onset_time', align_range=(1000, 1500), ignored_trials=~cond_mask)
    for idx, trial in cond_data.groupby('trial_id'):
        ax_3.plot(trial.hand_pos.x, trial.hand_pos.y, color=plt.cm.hsv(cond[1] / 360), linewidth=0.5)
        ax_3.set_xlim([-15, 15])
        ax_3.set_ylim([-15, 15])

    cond_data = dataset.make_trial_data(align_field='move_onset_time', align_range=(1500, 2000), ignored_trials=~cond_mask)
    # Plot reaches on appropriate subplot
    for idx, trial in cond_data.groupby('trial_id'):
        ax_4.plot(trial.hand_pos.x, trial.hand_pos.y, color=plt.cm.hsv(cond[1] / 360), linewidth=0.5)
        ax_4.set_xlim([-15, 15])
        ax_4.set_ylim([-15, 15])

            
# Add labels
ax_0.set_title('-500 to 0')
ax_1.set_title('0 to 500')
ax_2.set_title('500 to 1000')
ax_3.set_title('1000 to 1500')
ax_4.set_title('1500 to 2000')



ax_0.axis("off")
ax_1.axis("off")
ax_2.axis("off")
ax_3.axis("off")
ax_4.axis("off")

figDir = '/Users/sherryan/area2_population_analysis/sfn/'
plt.tight_layout()
# plt.savefig(figDir + monkey + '_active_traj.pdf',dpi = 'figure')
# plt.suptitle('Active Reach Trajectories')
plt.show()

## Smoothing effect

In [ ]:
#number of neurons
n_neurons = dataset.data.spikes.shape[1]
print(n_neurons)

In [ ]:
# number of useable active trials
# Filter out invalid trials (labeled 'none') and trials in other conditions
active_mask = (np.all(dataset.trial_info[['ctr_hold_bump']] == False, axis=1)) & \
    (dataset.trial_info.split != 'none')
active_data = dataset.make_trial_data(align_field='move_onset_time', align_range=(-100, 500), ignored_trials=~active_mask)
n_trials = active_data['trial_id'].nunique();
print(n_trials)

In [ ]:
#number of time bins per trial = align_range / bin_size
for idx, trial in active_data.groupby('trial_id'):
    n_timepoints = trial.shape[0]
    break
print(n_timepoints)

In [ ]:
dataset.smooth_spk(40, name='smth_40')
dataset.smooth_spk(25, name='smth_25')
dataset.smooth_spk(10, name='smth_10')

active_trials_idx_array = np.empty((n_trials,1))
raw_active_trials_array = np.empty((n_trials,n_timepoints,n_neurons))
active_trials_smth_10_array = np.empty((n_trials,n_timepoints,n_neurons))
active_trials_smth_25_array = np.empty((n_trials,n_timepoints,n_neurons))
active_trials_smth_40_array = np.empty((n_trials,n_timepoints,n_neurons))
i = 0
for idx, trial in active_data.groupby('trial_id'):
    active_trials_idx_array[i,:]=idx
    raw_active_trials_array[i,:,:]=trial.spikes.to_numpy()
    active_trials_smth_10_array[i,:,:]=trial.spikes_smth_10.to_numpy()
    active_trials_smth_25_array[i,:,:]=trial.spikes_smth_25.to_numpy()
    active_trials_smth_40_array[i,:,:]=trial.spikes_smth_40.to_numpy()
    i+=1

In [ ]:
print(active_trials_smth_40_array.shape)
print(active_trials_idx_array.shape)

In [ ]:
x_axis = np.arange(-500,700,5)
plt.plot(x_axis,raw_active_trials_array[0,:,10]/dataset.bin_width*1000,label = 'raw')
plt.plot(x_axis,active_trials_smth_10_array[0,:,10]/dataset.bin_width*1000 ,label='smooth_10')
plt.plot(x_axis,active_trials_smth_25_array[0,:,10]/dataset.bin_width*1000 ,label='smooth_25')
plt.plot(x_axis,active_trials_smth_40_array[0,:,10]/dataset.bin_width*1000 ,label='smooth_40')
plt.title('Spike smoothing (5ms bin)')
plt.ylabel('Firing rate (spks/s)')
plt.xlabel('time (ms)')
plt.legend()